In [7]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from copy import deepcopy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import RandomSampler
import matplotlib.pyplot as plt

### GPU运行工具，后面会用

In [2]:
device = torch.device('cuda')

### 创建根目录，利用跟目录找到其他目录

In [3]:
src = '/Users/ryo/Desktop/competition_data'
save_weight_path = src + '/weight'
train_image_dir = src + '/train/images'
train_mask_dir = src + '/train/masks'
test_image_dir = src + '/test/images'

1. 读取depth数据
2. 按z的大小排序，替换
3. 删除z列，替换
4. 分组


In [4]:
depths = pd.read_csv(src + '/depths.csv')
depths.sort_values('z', inplace=True)
depths.drop('z', axis=1, inplace=True)
depths['fold'] = (list(range(0,5)) * depths.shape[0])[:depths.shape[0]]

1. 读取train的数据
2. 合并数据
3. 创建空数列dist
4. 读取图片，灰度图（101，101）
5. 计算图片不同像素值个数
6. 存入dist

In [21]:
train_df = pd.read_csv(src + '/train.csv')
train_df = train_df.merge(depths)
dist = []

for id in train_df.id.values:
  img = plt.imread(train_image_dir+f'/{id}.png')
  dist.append(np.unique(img).shape[0])
train_df['unique_pixels'] = dist

1. 获取id列表
2. 根据fold对数据进行分组（5组）

In [22]:
all_id = train_df['id'].values
fold = []
for i in range(5):
  fold.append(train_df.loc[train_df['fold']==i, 'id'].values)

## 扩张
|函数|作用|描述|
|-|-|-|
|ConvTranspose2d|扩张，倍数与stride有关|输入有限制|

In [25]:
ConvTranspose2d = nn.ConvTranspose2d(64,32,2,2)
input  = torch.Tensor(64,64,32,32)
print('input',input.shape)
output = ConvTranspose2d(input)
print('output',output.shape)

input torch.Size([64, 64, 32, 32])
output torch.Size([64, 32, 64, 64])


## 归一化
|函数|作用|描述|
|-|-|-|
|BatchNorm2d|归一化，防止梯度消失或则过大|输入有限制，且对输出维度不影响|

In [27]:
BatchNorm2d = nn.BatchNorm2d(num_features=64)
input  = torch.Tensor(64,64,32,32)
print('input',input.shape)
output = BatchNorm2d(input)
print('output',output.shape)

input torch.Size([64, 64, 32, 32])
output torch.Size([64, 64, 32, 32])


## 最大池化
|函数|作用|描述|
|-|-|-|
|BatchNorm2d|最大池化，缩小倍数与stride有关，防止过拟合|输入无限制，|

In [29]:
BatchNorm2d = nn.MaxPool2d(kernel_size=2,stride=2,dilation=1)
input  = torch.Tensor(64,64,32,32)
print('input',input.shape)
output = BatchNorm2d(input)
print('output',output.shape)

input torch.Size([64, 64, 32, 32])
output torch.Size([64, 64, 16, 16])


## 卷积
|函数|作用|描述|
|-|-|-|
|Conv2d|卷积层，增加数据深度，<br>相比于全连接参数更少，适合更深的模型|输入有限制，<br>参数调节输出深度，以及长度宽度|

In [32]:
Conv2d= nn.Conv2d(in_channels=1,out_channels=8,kernel_size=7,stride=2,padding=2)
input  = torch.Tensor(64,1,32,32)
print('input',input.shape)
output = Conv2d(input)
print('output',output.shape)

input torch.Size([64, 1, 32, 32])
output torch.Size([64, 8, 15, 15])
